In [2]:
# import numpy as np
# import cv2 as cv
# cap = cv.VideoCapture(0)
# if not cap.isOpened():
#     print("Cannot open camera")
#     exit()
# while True:
#     # Capture frame-by-frame
#     ret, frame = cap.read()
#     # if frame is read correctly ret is True
#     if not ret:
#         print("Can't receive frame (stream end?). Exiting ...")
#         break
#     # Our operations on the frame come here
#     gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
#     # Display the resulting frame
#     cv.imshow('frame', gray)
#     if cv.waitKey(1) == ord('q'):
#         break
# # When everything done, release the capture
# cap.release()
# cv.destroyAllWindows()

In [2]:
# Data Processing and model code
# import necessary packages

import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
import time
import datetime
from pathlib import Path

# initialize mediapipe
mpHands = mp.solutions.hands
mpFace = mp.solutions.face_detection
hands = mpHands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh


# Load the gesture recognizer model
# model = load_model('mp_hand_gesture')

# Load class names
# f = open('gesture.names', 'r')
# classNames = f.read().split('\n')
# f.close()
# print(classNames)
image_dir = "wiki_crop/00"

# Initialize the webcam
drawing_spec = mpDraw.DrawingSpec(thickness=1, circle_radius=1)

files = list(enumerate(Path(image_dir).glob('*')))

# display(files)
frame_index = 0

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    while True:
        frame = cv2.imread(str(files[frame_index][1]), -1)
        
        if not (np.any(frame)):
          print("Ignoring empty image.")
          # If loading a video, use 'break' instead of 'continue'.
          continue
        # Read each frame from the webcam

#         display(frame)
        x, y, c = frame.shape

        # Flip the frame vertically
        frame.flags.writeable = False
#         frame = cv2.flip(frame, 2)
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get hand landmark prediction
#         result = hands.process(frame)
        resultFace = face_mesh.process(frame)
        # print(result)

#         className = ''

        # post process the result
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
#         if result.multi_hand_landmarks:
#             landmarks = []
#             count = 0
#             for handslms in result.multi_hand_landmarks:
#                 count +=1
#                 for lm in handslms.landmark:
#                     # print(id, lm)
#                     lmx = int(lm.x * x)
#                     lmy = int(lm.y * y)

#                     landmarks.append([lmx, lmy])
                    

#                 # Drawing landmarks on frames
#                 mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)
#                 if (count <= 1):
#                     # Predict gesture
#                     prediction = model.predict([landmarks])
#                     # print(prediction)
#                     classID = np.argmax(prediction)
#                     className = classNames[classID]
#                 print("hands landmarks: ")
#                 display(handslms)
#         468 face mesh points
        if resultFace.multi_face_landmarks:
            for face_landmarks in resultFace.multi_face_landmarks:
                mpDraw.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
                mpDraw.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mpDraw.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
            print("face landmarks: ")
#             display(face_landmarks)
            current_time = datetime.datetime.now(datetime.timezone.utc)
            xyzt = [(lm.x, lm.y, lm.z, current_time) for lm in face_landmarks.landmark]
#             face_lm_inputs = 
#             display(xyzt)
            
#         time.sleep(5)
            
        # show the prediction on the frame
#         frame = cv2.flip(frame, 1)
#         cv2.putText(frame, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
#                        1, (0,0,255), 2, cv2.LINE_AA)
        
        # Show the final output
        cv2.imshow("Output", frame) 

        if cv2.waitKey(1) == ord('q'):
            break
        elif cv2.waitKey(1) == ord('n'):
            if frame_index < len(files):
                frame_index =+ 1
            else:
                frame_index = 0
        elif cv2.waitKey(1) == ord('p'):
            if frame_index > 0:
                frame_index =- 1
            else:
                frame_index = len(files)
        

# release the webcam and destroy all active windows
# cap.release()

cv2.destroyAllWindows()

face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks

face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks

face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks

face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks

face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks

...

face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks

face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks

face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks

face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks

face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks: 
face landmarks

In [9]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
import time
import datetime
from pathlib import Path
import matplotlib.pyplot as plt

# initialize mediapipe
mpHands = mp.solutions.hands
mpFace = mp.solutions.face_detection
hands = mpHands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh


# Load the gesture recognizer model
# model = load_model('mp_hand_gesture')

# Load class names
# f = open('gesture.names', 'r')
# classNames = f.read().split('\n')
# f.close()
# print(classNames)
image_dir = "wiki_crop/00"

# Initialize the webcam
# drawing_spec = mpDraw.DrawingSpec(thickness=1, circle_radius=1)
# cap = cv2.VideoCapture(0)
files = list(enumerate(Path(image_dir).glob('*')))
frame_index = 300
frame = cv2.imread(str(files[frame_index][1]), -1)

# plt.imshow(frame, cmap='gray')
cv2.imshow("Output", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()